In [17]:
!pip install simpletransformers

In [18]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging

In [19]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [20]:
data=pd.read_csv('IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [21]:
from sklearn.model_selection import train_test_split

data['labels']=data['sentiment'].map({'positive':1,'negative':0})

train_df, eval_df=train_test_split(data[['review', 'labels']], test_size=0.2, random_state=42)

In [22]:
train_df.columns=["text", "labels"]
eval_df.columns=["text", "labels"]

In [23]:
train_df.head()

,text,labels
39087,That's what I kept asking myself during the ma...,0
30893,I did not watch the entire movie. I could not ...,0
45278,A touching love story reminiscent of In the M...,1
16398,This latter-day Fulci schlocker is a totally a...,0
13653,"First of all, I firmly believe that Norwegian ...",0


In [24]:
eval_df.head()

,text,labels
33553,I really liked this Summerslam due to the look...,1
9427,Not many television shows appeal to quite as m...,1
199,The film quickly gets to a major chase scene w...,0
12447,Jane Austen would definitely approve of this o...,1
39489,Expectations were somewhat high for me when I ...,0


In [25]:
model_args=ClassificationArgs(num_train_epochs=1, output_dir='./fine_tuned')

model=ClassificationModel(
    'roberta',
    'roberta-base',
    num_labels=2,
    use_cuda=True,
    args=model_args
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
# Fine Tuning Roberta on the training data
model.train_model(train_df)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/80 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/5000 [00:00<?, ?it/s]

(5000, 0.37703188428878787)

In [27]:
# Evaluating the fine tuned model on the Testing Data
results, model_outputs, wrong_predictions=model.eval_model(eval_df)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/20 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]

In [33]:
# Making predictions using the fine tuned model
predictions, raw_outputs=model.predict("Worst movie")

predictions

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1]

In [37]:
model = ClassificationModel("roberta", "fine_tuned", use_cuda=True)

text_to_classify = ["I love this movie!", "This was a terrible film.", "Worst Movie","Waste of time"]
predictions, raw_outputs = model.predict(text_to_classify)


for i in predictions:
  if i==1:
    print("Positive")
  else:
    print("Negative")

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Positive
Negative
Negative
Negative
